In [ ]:
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0) #above was used to generate xlsx format Excel file
RNA_code='ACGU' 
interval=3 # RNA_code and interval used mix used to generate AAA AAC AAG ...
division_num=10
divided_num=np.float(division_num)# ten fold so the result should divided by ten


# seq=[e.replace('U','A') for e in seq]
# seq=[e.replace('G','C') for e in seq]
seq=pd.read_csv('/home02/chenhuangrong/'+name+'.csv',header=None,index_col=False)
seq=seq.values

positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]



#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end


#shuffle the data of positive and negative begining
kf = KFold(n_splits=division_num,shuffle=False)  # positive and negative samples will be shuffled 
for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
#shuffle the data of positive and negative end


    #generate train and test data begining
    X_train = np.array(positive_negative_x_train)
    Y_train = list(map(lambda x: 1, xrange(len(X_train) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(X_train) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(positive_negative_y_train)
    Y_test  = list(map(lambda x: 1, xrange(len(X_test) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(X_test) / 2)))
    Y_test.extend(Y2_test )
    Y_test = np.array(Y_test)
    #generate train and test data end

    
    # training model and optimized parameters of C and gamma begining
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    # training model and optimized parameters of C and gamma end
    
    
    #print best C and gamma begining
    C=clf.best_params_['C']
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    #print best C and gamma end
    
    
    #getting predict probability and predict label begining
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    #getting predict probability and predict label begining

    #the process of generating  usefule data begining
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)

    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
    #the process of generating  usefule data end
    
#the process of save  data to disk begining   
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
# pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PseEIIP%d.csv'%select_num,header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[str(X_train.shape[1]),"正："+str(pos_all)+'负：'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all]
final_out_to_excel.append(savedata)

print savedata
pd.DataFrame(final_out_to_excel).to_excel('/home02/chenhuangrong/'+name+'.xlsx',sheet_name="independent_test",index=False,header=False)
#the process of save  data to disk end